In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
with open('ukraine.json', 'r') as f:
    ukraine = json.load(f)

pop = pd.read_csv('population_trends.csv')
pop_grouped = pop.copy()
pop_grouped['rate'][pop_grouped['year'] != pop.year.max()] = 0

C:\Users\328F~1\AppData\Local\Temp/ipykernel_27352/4179961822.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pop_grouped['rate'][pop_grouped['year'] != pop.year.max()] = 0


In [3]:
for i in ukraine['features']:
    i['id'] = i['properties']['NAME_1']

In [4]:
pop_grouped = pop_grouped.groupby('region').last().reset_index()

In [5]:
regions = pop.region.unique()
years  = pop.year.unique()

In [6]:
first = np.array([13,17,144])
eight = np.array([240,233,186])
last = np.array([193,0,0])

In [7]:
step = ((last - eight) / 95 * 1).astype(int)

In [8]:
colorscale = [[150,150,150], [0,17,144], [120,120,186], [240,233,186]]
for i in range(93):
    colorscale.append((eight + (step * (i+1))).tolist())
colorscale = [", ".join([str(j) for j in i]) for i in colorscale]
colorscale = [f"rgb({i})" for i in colorscale]

In [9]:
def update_trace_from_line(trace, points, selector):
    if len(points.point_inds) == 0:
        return
    
    for i in range(1, len(fig.data) ):
        if fig.data[i].name == trace.name:
            fig.data[i].line.width += 1
        else:
            fig.data[i].line.width = 0.4

    ops = [0.6 for i in fig.data[0].locations]
    ops[fig.data[0].locations.tolist().index(trace.name)] = 0.99
    fig.data[0].marker.opacity = ops

In [10]:
def update_trace_from_map(trace, points, selector):
    if len(points.point_inds) == 0:
        return
    ind = points.point_inds[0]
    region = fig.data[0].locations.tolist()[ind]
    
    for i in range(1, len(fig.data) ):
        if fig.data[i].name == region:
            fig.data[i].line.width += 1
        else:
            fig.data[i].line.width = 0.4
    
    ops = [0.6 for i in fig.data[0].locations]
    ops[fig.data[0].locations.tolist().index(region)] = 0.99
    fig.data[0].marker.opacity = ops

In [11]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

fig = go.FigureWidget(make_subplots(
    rows=1, cols=2,
    column_widths=[0.6, 0.4],
    row_heights=[0.4],
    specs=[[{"type": "choroplethmapbox", "rowspan": 1}, {"type": "xy"}]]))

fig.add_trace(go.Choroplethmapbox(geojson=ukraine, locations = pop_grouped.region, z=pop_grouped.rate, colorbar_title = "Rate",
                                    colorscale=colorscale[::-1], zmin=pop_grouped.rate.min(), zmax=0,
                                    marker_opacity=0.4, marker_line_width=0, name="Map"),row=1, col=1)
fig.update_layout(mapbox_style="carto-darkmatter",
                  mapbox_zoom=4, mapbox_center = {"lat": 48.3794, "lon": 31.1656}, coloraxis_colorbar_x=-2)
fig.update_layout(margin={"r":0,"l":0, "t":50 ,"b":0})
fig.data[0].colorbar.x=0
for region in regions:
    y = pop[pop['region'] == region]['rate'].to_numpy()
    fig.add_trace(go.Scatter(x=years, y=y,
                        mode='lines', hovertemplate = "Rate: %{y}",
                        name=region,line_width=0.4),row=1, col=2)
fig.update_layout(height=600, width=1500, title_text="Population Change Rate of Ukrainian Regions for 2019",
    yaxis_title="Rate",
                  template="plotly_dark", colorscale_diverging=px.colors.qualitative.Light24)

for i in range(1, len(fig.data) ):
    fig.data[i].on_hover(update_trace_from_line)

    
fig.data[0].on_hover(update_trace_from_map)
fig.data[0].marker.opacity = [0.99 for i in fig.data[0].locations]

fig.data[0].colorbar.ticks = "outside"
fig

FigureWidget({
    'data': [{'colorbar': {'ticks': 'outside', 'title': {'text': 'Rate'}, 'x': 0},
            …

In [13]:
# !jupyter nbconvert --to html main.ipynb

[NbConvertApp] Converting notebook main.ipynb to html
[NbConvertApp] Writing 603462 bytes to main.html
